In [2]:
import pandas as pd
import os

path = r'/home/students/mat/a/al417707/NYPD_projekt/GDP/'

In [3]:
GDP1 = pd.read_csv(r'GDP1.csv', skiprows = 4)
GDP2 = pd.read_csv(r'Metadata_Country_API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4751562.csv')
GDP3 = pd.read_csv(r'Metadata_Indicator_API_NY.GDP.MKTP.CD_DS2_en_csv_v2_4751562.csv')

Pop1 = pd.read_csv(r'API_SP.POP.TOTL_DS2_en_csv_v2_4751604.csv', skiprows = 3)
Pop2 = pd.read_csv(r'Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2_4751604.csv')
Pop3 = pd.read_csv(r'Metadata_Indicator_API_SP.POP.TOTL_DS2_en_csv_v2_4751604.csv')

co2 = pd.read_csv(r'fossil-fuel-co2-emissions-by-nation_csv.csv')


#czyszczenie danych
GDP1.drop('Unnamed: 66',axis = 1, inplace = True)
GDP2.drop('Unnamed: 5',axis = 1, inplace = True)

Pop1.drop('Unnamed: 66',axis = 1, inplace = True)
Pop2.drop('Unnamed: 5',axis = 1, inplace = True)

In [4]:
#wybór lat:
GDP_yrs = [int(x) for x in list(GDP1.columns[4:])]
Pop_yrs = [int(x) for x in list(Pop1.columns[4:])]
co2_yrs = [int(x) for x in list(co2['Year'])]

yrs = list(set(GDP_yrs).intersection(set(Pop_yrs)).intersection(set(co2_yrs)))

co2 = co2[co2.Year.isin(yrs)].reset_index(drop = True)
Pop1 = Pop1[list(Pop1.columns[:4])+[str(x)for x in yrs]]
GDP1 = GDP1[list(GDP1.columns[:4])+[str(x)for x in yrs]]

In [5]:
#merge tabel
Pop1_melted = Pop1.melt(id_vars = ['Country Name','Country Code','Indicator Name','Indicator Code'], var_name = 'Year', value_name = 'Population')
GDP1_melted = GDP1.melt(id_vars = ['Country Name','Country Code','Indicator Name','Indicator Code'], var_name = 'Year', value_name = 'GDP')

Dane = Pop1_melted.merge(GDP1_melted, on = ['Country Name','Country Code','Year'], suffixes = ['_Pop','_GDP'])
Dane.rename(columns = {'Country Name':'Country'}, inplace = True)
Dane = Dane.astype({'Year':'int64'})
Dane['Country'] = Dane['Country'].str.upper()
Dane = Dane.merge(co2, on = ['Country','Year'])
display(Dane)

In [121]:
#Analizy - co2 na mieszkańca
Dane['Co2/pop'] = Dane['Total']/Dane['Population']
#display(Dane.sort_values('Co2/pop').dropna())

#jakie kraje średnio mają największą emisję? Wybieramy 5 topowych
Co2_avg = Dane.groupby('Country').agg({'Co2/pop':'mean'}).sort_values('Co2/pop')
wybrane_kraje = Co2_avg.index
wybrane_kraje = wybrane_kraje[-5:]
wybrane_kraje

#tworzę tabelkę - wybrane kraje z największą emisją posortowane po latach
co2_top5 = Dane[Dane['Country'].isin(wybrane_kraje)]
co2_top5 = co2_top5[['Country','Year','Total','Co2/pop']].sort_values('Year')
display(co2_top5)

,Country,Year,Total,Co2/pop
3,UNITED ARAB EMIRATES,1960,3,0.000032
61,KUWAIT,1960,2128,0.007910
65,LUXEMBOURG,1960,3141,0.010004
91,QATAR,1960,48,0.001013
117,UNITED ARAB EMIRATES,1961,3,0.000030
...,...,...,...,...
7647,LUXEMBOURG,2014,2634,0.004735
7567,UNITED ARAB EMIRATES,2014,57641,0.006256
7595,CURACAO,2014,1604,0.010288
7640,KUWAIT,2014,26018,0.007049


In [114]:
#Analizy - co2 na mieszkańca V2 (chyba poprawne)
Dane['Co2/pop'] = Dane['Total']/Dane['Population']

co2_top_yearly = Dane.set_index('Country').groupby(['Year'])['Co2/pop'].nlargest(5)
co2_top_yearly = pd.DataFrame(co2_top_yearly).reset_index()
co2_top_yearly = co2_top_yearly.merge(Dane[['Year','Country','Total']])
display(co2_top_yearly)

,Year,Country,Co2/pop,Total
0,1960,LUXEMBOURG,0.010004,3141
1,1960,KUWAIT,0.007910,2128
2,1960,UNITED KINGDOM,0.003041,159340
3,1960,NEW CALEDONIA,0.002975,235
4,1960,CANADA,0.002937,52603
...,...,...,...,...
270,2014,QATAR,0.011960,29412
271,2014,CURACAO,0.010288,1604
272,2014,TRINIDAD AND TOBAGO,0.009263,12619
273,2014,KUWAIT,0.007049,26018


In [9]:
#Analizy - kraje z największym przychodem na mieszkańca 
Dane['GDP_PC'] = Dane['GDP']/Dane['Population']
GDP_PC_avg = Dane.groupby('Country').agg({'GDP_PC':'mean'}).sort_values('GDP_PC')
wybrane_kraje_gdp = GDP_PC_avg.dropna().index[-5:]

GDP_PC_top5 = Dane[Dane['Country'].isin(wybrane_kraje_gdp)]
GDP_PC_top5 = GDP_PC_top5[['Country','Year','GDP','GDP_PC']].sort_values('Year').dropna()
display(GDP_PC_top5)

,Country,Year,GDP,GDP_PC
13,BERMUDA,1960,8.446665e+07,1902.402119
18,SWITZERLAND,1960,9.522747e+09,1787.360348
65,LUXEMBOURG,1960,7.039257e+08,2242.015817
127,BERMUDA,1961,8.924999e+07,1961.538169
132,SWITZERLAND,1961,1.071271e+10,1971.316323
...,...,...,...,...
7643,LIECHTENSTEIN,2014,6.657171e+09,178864.851914
7582,BERMUDA,2014,6.413988e+09,98467.683994
7589,SWITZERLAND,2014,7.343966e+11,89684.707580
7596,CAYMAN ISLANDS,2014,4.563018e+09,74990.432744


In [115]:
#Analizy - kraje z największym przychodem na mieszkańca V2
Dane['GDP_PC'] = Dane['GDP']/Dane['Population']

GDP_top_yearly = Dane.set_index('Country').groupby(['Year'])['GDP_PC'].nlargest(5)
GDP_top_yearly = pd.DataFrame(GDP_top_yearly).reset_index()
GDP_top_yearly = GDP_top_yearly.merge(Dane[['Year','Country','GDP']])
display(GDP_top_yearly)

,Year,Country,GDP_PC,GDP
0,1960,NEW ZEALAND,2312.949992,5.485855e+09
1,1960,CANADA,2259.294285,4.046172e+10
2,1960,LUXEMBOURG,2242.015817,7.039257e+08
3,1960,SWEDEN,2114.002973,1.582259e+10
4,1960,BERMUDA,1902.402119,8.446665e+07
...,...,...,...,...
270,2014,LIECHTENSTEIN,178864.851914,6.657171e+09
271,2014,LUXEMBOURG,123678.702143,6.880481e+10
272,2014,BERMUDA,98467.683994,6.413988e+09
273,2014,NORWAY,97019.182753,4.984101e+11


In [63]:
#Analizy - różnica w ostatnich 10 latach

co2_2004 = Dane[Dane['Year']==2004][['Country','Total']]
co2_2014 = Dane[Dane['Year']==2014][['Country','Total']]

kraje = list(set(co2_2004.Country) & set(co2_2014.Country))
co2_2004 = co2_2004[co2_2004['Country'].isin(kraje)].sort_values('Country').reset_index()
co2_2014 = co2_2014[co2_2014['Country'].isin(kraje)].sort_values('Country').reset_index()

co2_diff = pd.DataFrame()
co2_diff['Country'] = co2_2014['Country']
co2_diff['Diff'] = co2_2014['Total'] - co2_2004['Total']
co2_diff = co2_diff.sort_values('Diff')

diff_lowest = co2_diff[:5].Country
diff_highest = co2_diff[-5:].Country

In [65]:
diff_highest

119    RUSSIAN FEDERATION
67              INDONESIA
21                 BRAZIL
122          SAUDI ARABIA
66                  INDIA
Name: Country, dtype: object

In [57]:
co2_diff

,Country,Diff
149,UNITED KINGDOM,-32566
147,UKRAINE,-31660
131,SPAIN,-28757
53,GERMANY,-26430
72,JAPAN,-14170
...,...,...
119,RUSSIAN FEDERATION,28151
67,INDONESIA,34508
21,BRAZIL,52354
122,SAUDI ARABIA,55962


In [11]:
display(Dane)

,Country,Country Code,Indicator Name_Pop,Indicator Code_Pop,Year,Population,Indicator Name_GDP,Indicator Code_GDP,GDP,Total,Solid Fuel,Liquid Fuel,Gas Fuel,Cement,Gas Flaring,Per Capita,Bunker fuels (Not in Total),Co2/pop,GDP_PC
0,AFGHANISTAN,AFG,"Population, total",SP.POP.TOTL,1960,8996967.0,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,113,35,74,0,5,0,0.01,0,0.000013,59.773234
1,ANGOLA,AGO,"Population, total",SP.POP.TOTL,1960,5454938.0,GDP (current US$),NY.GDP.MKTP.CD,NaN,150,34,95,0,22,0,0.03,41,0.000027,NaN
2,ALBANIA,ALB,"Population, total",SP.POP.TOTL,1960,1608800.0,GDP (current US$),NY.GDP.MKTP.CD,NaN,552,89,430,23,10,0,0.34,0,0.000343,NaN
3,UNITED ARAB EMIRATES,ARE,"Population, total",SP.POP.TOTL,1960,92417.0,GDP (current US$),NY.GDP.MKTP.CD,NaN,3,0,3,0,0,0,0.03,0,0.000032,NaN
4,ARGENTINA,ARG,"Population, total",SP.POP.TOTL,1960,20481781.0,GDP (current US$),NY.GDP.MKTP.CD,NaN,13312,1066,11242,645,359,0,0.65,211,0.000650,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7716,VANUATU,VUT,"Population, total",SP.POP.TOTL,2014,263888.0,GDP (current US$),NY.GDP.MKTP.CD,7.723157e+08,42,0,42,0,0,0,0.16,10,0.000159,2926.679960
7717,SAMOA,WSM,"Population, total",SP.POP.TOTL,2014,192220.0,GDP (current US$),NY.GDP.MKTP.CD,7.568060e+08,54,0,54,0,0,0,0.28,9,0.000281,3937.186298
7718,SOUTH AFRICA,ZAF,"Population, total",SP.POP.TOTL,2014,54544184.0,GDP (current US$),NY.GDP.MKTP.CD,3.811989e+11,133562,113068,16448,2404,1642,0,2.47,2945,0.002449,6988.808739
7719,ZAMBIA,ZMB,"Population, total",SP.POP.TOTL,2014,15399793.0,GDP (current US$),NY.GDP.MKTP.CD,2.714102e+10,1228,132,797,0,299,0,0.08,33,0.000080,1762.427817


In [12]:
display(co2)
display(Pop1)
display(GDP1)

,Year,Country,Total,Solid Fuel,Liquid Fuel,Gas Fuel,Cement,Gas Flaring,Per Capita,Bunker fuels (Not in Total)
0,1960,AFGHANISTAN,113,35,74,0,5,0,0.01,0
1,1960,ALBANIA,552,89,430,23,10,0,0.34,0
2,1960,ALGERIA,1680,343,1193,0,144,0,0.16,438
3,1960,ANGOLA,150,34,95,0,22,0,0.03,41
4,1960,ANTIGUA & BARBUDA,10,0,10,0,0,0,0.18,0
...,...,...,...,...,...,...,...,...,...,...
10875,2014,VIET NAM,45517,19246,12694,5349,8229,0,0.49,761
10876,2014,WALLIS AND FUTUNA ISLANDS,6,0,6,0,0,0,0.44,1
10877,2014,YEMEN,6190,137,5090,581,381,0,0.24,153
10878,2014,ZAMBIA,1228,132,797,0,299,0,0.08,33


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,100028.0,100830.0,101226.0,101362.0,101452.0,101665.0,102050.0,102565.0,103165.0,103776.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,453404076.0,465581372.0,478166911.0,491173160.0,504604672.0,518468229.0,532760424.0,547482863.0,562601578.0,578075373.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,25654274.0,26433058.0,27100542.0,27722281.0,28394806.0,29185511.0,30117411.0,31161378.0,32269592.0,33370804.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,305520588.0,313985474.0,322741656.0,331772330.0,341050537.0,350556886.0,360285439.0,370243017.0,380437896.0,390882979.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,19433604.0,20149905.0,20905360.0,21695636.0,22514275.0,23356247.0,24220660.0,25107925.0,26015786.0,26941773.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,"Population, total",SP.POP.TOTL,947000.0,966000.0,994000.0,1022000.0,1050000.0,1078000.0,...,1705780.0,1719536.0,1733404.0,1747383.0,1761474.0,1775680.0,1791000.0,1807106.0,1818117.0,1812771.0
262,"Yemen, Rep.",YEM,"Population, total",SP.POP.TOTL,5315351.0,5393034.0,5473671.0,5556767.0,5641598.0,5727745.0,...,20107416.0,20687648.0,21282514.0,21892149.0,22516464.0,23154854.0,23807586.0,24473176.0,25147112.0,25823488.0
263,South Africa,ZAF,"Population, total",SP.POP.TOTL,17099836.0,17524533.0,17965733.0,18423157.0,18896303.0,19384838.0,...,47880595.0,48489464.0,49119766.0,49779472.0,50477013.0,51216967.0,52003759.0,52832659.0,53687125.0,54544184.0
264,Zambia,ZMB,"Population, total",SP.POP.TOTL,3070780.0,3164330.0,3260645.0,3360099.0,3463211.0,3570466.0,...,11856244.0,12173518.0,12502958.0,12848531.0,13215142.0,13605986.0,14023199.0,14465148.0,14926551.0,15399793.0


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.359777e+09,2.469832e+09,2.677654e+09,2.843017e+09,2.553631e+09,2.453631e+09,2.637989e+09,2.615084e+09,2.727933e+09,2.791061e+09
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2.129059e+10,2.180847e+10,2.370702e+10,2.821004e+10,2.611879e+10,2.968217e+10,...,5.122111e+11,5.759214e+11,6.611793e+11,7.082869e+11,7.192167e+11,8.604783e+11,9.644185e+11,9.730435e+11,9.839370e+11,1.003679e+12
2,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,6.209138e+09,6.971286e+09,9.747880e+09,1.010931e+10,1.241616e+10,1.585668e+10,1.780511e+10,1.990732e+10,2.014640e+10,2.049713e+10
3,Africa Western and Central,AFW,GDP (current US$),NY.GDP.MKTP.CD,1.040414e+10,1.112789e+10,1.194319e+10,1.267633e+10,1.383837e+10,1.486223e+10,...,3.105575e+11,3.933052e+11,4.617914e+11,5.664808e+11,5.070436e+11,5.915958e+11,6.709830e+11,7.275704e+11,8.207927e+11,8.649905e+11
4,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.697092e+10,5.238101e+10,6.526645e+10,8.853861e+10,7.030717e+10,8.169956e+10,1.094365e+11,1.249982e+11,1.334016e+11,1.372444e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.181777e+09,5.015895e+09,5.344014e+09,6.341737e+09,6.163785e+09,6.735731e+09,7.074658e+09
262,"Yemen, Rep.",YEM,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.674634e+10,1.906198e+10,2.165053e+10,2.691085e+10,2.513027e+10,3.090675e+10,3.272642e+10,3.540134e+10,4.041524e+10,4.322859e+10
263,South Africa,ZAF,GDP (current US$),NY.GDP.MKTP.CD,8.748597e+09,9.225996e+09,9.813996e+09,1.085420e+10,1.195600e+10,1.306899e+10,...,2.888685e+11,3.038609e+11,3.330755e+11,3.161321e+11,3.297530e+11,4.173651e+11,4.582015e+11,4.344005e+11,4.008860e+11,3.811989e+11
264,Zambia,ZMB,GDP (current US$),NY.GDP.MKTP.CD,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,1.082857e+09,...,8.331870e+09,1.275686e+10,1.405696e+10,1.791086e+10,1.532834e+10,2.026556e+10,2.345952e+10,2.550306e+10,2.803724e+10,2.714102e+10
